In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def softmax(t):
    res = np.exp(t)
    return res / np.sum(res)

def relu(t):
    return np.maximum(t, 0)

def relu_deriv(t):
    return (t>=0).astype(float)

def to_one_hot_encoding(y, dim = 10):
    res = np.zeros([1, dim])
    res[0][y] = 1
    return res


class Perceptron:
    def __init__(self, input_dim, h1_dim, h2_dim, out_dim, learn_rate = 0.005):
        self.input_dim = input_dim
        self.h1_dim = h1_dim
        self.h2_dim = h2_dim
        self.out_dim = out_dim
        self.learn_rate = learn_rate

        self.t1 = np.zeros([1, self.h1_dim])
        self.h1 = np.zeros([1, self.h1_dim])
        self.t2 = np.zeros([1, self.h2_dim])
        self.h2 = np.zeros([1, self.h2_dim])
        self.t3 = np.zeros([1, self.out_dim])
        self.out = np.zeros([1, self.out_dim])

        self.w1 = np.random.rand(self.input_dim, self.h1_dim) - 0.5
        self.b1 = np.random.rand(1, self.h1_dim) - 0.5
        self.w2 = np.random.rand(self.h1_dim, self.h2_dim) - 0.5
        self.b2 = np.random.rand(1, self.h2_dim) - 0.5
        self.w3 = np.random.rand(self.h2_dim, self.out_dim) - 0.5
        self.b3 = np.random.rand(1, self.out_dim) - 0.5
        self.drop_gradients()


    def drop_gradients(self):
        self.dE_dw1 = np.zeros((self.input_dim, self.h1_dim))
        self.dE_db1 = np.zeros((1, self.h1_dim))
        self.dE_dw2 = np.zeros((self.h1_dim, self.h2_dim))
        self.dE_db2 = np.zeros((1, self.h2_dim))
        self.dE_dw3 = np.zeros((self.h2_dim, self.out_dim))
        self.dE_db3 = np.zeros((1, self.out_dim))


    def run(self, data, activation_function):
        self.t1 = data @ self.w1 + self.b1
        self.h1 = activation_function(self.t1)
        self.t2 = self.h1 @ self.w2 + self.b2
        self.h2 = activation_function(self.t2)
        self.t3 = self.h2 @ self.w3 + self.b3
        self.out = self.t3
        return self.out


    def calculate_gradient(self, data, right_out, activation_deriv):
        dE_dt3 = self.out - right_out
        self.dE_dw3 += self.h2.T @ dE_dt3
        self.dE_db3 = dE_dt3

        dE_dh2 = dE_dt3 @ self.w3.T
        dE_dt2 = dE_dh2 * activation_deriv(self.t2)
        self.dE_dw2 += self.h1.T @ dE_dt2
        self.dE_db2 += dE_dt2

        dE_dh3 = dE_dt2 @ self.w2.T
        dE_dt3 = dE_dh3 * activation_deriv(self.t1)
        self.dE_dw1 += data.T @ dE_dt3
        self.dE_db1 += dE_dt3


    def apply_gradient(self, batch_size=1):
        self.w1 -= self.learn_rate * self.dE_dw1 / batch_size
        self.w2 -= self.learn_rate * self.dE_dw2 / batch_size
        self.b1 -= self.learn_rate * self.dE_db1 / batch_size
        self.b2 -= self.learn_rate * self.dE_db2 / batch_size
        self.b3 -= self.learn_rate * self.dE_db3 / batch_size

        self.drop_gradients()



In [ ]:
def train_model(model, X_train, y_train, X_test, y_test):
    for epoch in range (1, 3):
        loss = 0
        for i in range(0, len(X_train)):
            prediction = model.run(data=X_train[i].flatten(), activation_function=relu)
            loss += np.sum((y_train[i] - prediction))/ np.sum(y_train[i])
            model.calculate_gradient(data=(X_train[i]).reshape(1,100*100), right_out=y_train[i], activation_deriv=relu_deriv)
            if i % 20 == 0:
                model.apply_gradient()

        test_loss = 0
        # for i in range(0, 10000):
        #     prediction = np.argmax(model.run(data=X_test[i], activation_function=relu))
        #     test_loss += (y_test[i] - prediction) ** 2
        print(f"Epoch №{epoch} finished with accuracy {round(loss/len(X_train) * 100, 2)}% Test dataset accuracy {round(test_loss/len(X_train) * 100, 2)}")


In [ ]:
X_train = np.load("data/test/test_x.npy")
y_train = np.load("data/test/test_y.npy")

In [ ]:
np.random.seed(0)
model = Perceptron(input_dim=100*100, h1_dim=100, h2_dim=20, out_dim=4)

In [ ]:
train_model(model, X_train, y_train, X_train, y_train)

In [ ]:
i = 9009
right_answer = model.run(data=X_train[i].flatten(), activation_function=relu)
print(right_answer, y_train[i])
fig = plt.figure
plt.imshow(X_train[i], origin='lower', cmap='gray')
plt.show()
